In [1]:
import sys
sys.path.append('/Users/pc/Documents/stfu/classes/_ee292d/CrowdCounting-P2PNet')
import os
import random
import torch
from collections import OrderedDict
from torch.utils.data import Dataset, Subset, DataLoader
from easydict import EasyDict
from PIL import Image
from torchvision import transforms
from models.backbone import BackboneBase_VGG
from models.matcher import build_matcher_crowd
from models.p2pnet import P2PNet, SetCriterion_Crowd
import models.vgg_ as vgg
from crowd_datasets.SHHA.SHHA import random_crop
from engine import train_one_epoch, evaluate_crowd_no_overlap, vis
import util.misc as utils
from tqdm import tqdm

SEED = 10541 #0x292D
VIS_DIR = 'vis'

In [2]:
def seed_all(seed=SEED):
    random.seed(seed)
    # np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [3]:
# need to load pretraining from elsewhere
class CustomVGGBackboneWrapper(BackboneBase_VGG):
    def __init__(self, name, return_interm_layers=True):
        if name == 'vgg16_bn':
            backbone = vgg.vgg16_bn()
        elif name == 'vgg16':
            backbone = vgg.vgg16()
        num_channels = 256
        super().__init__(backbone, num_channels, name, return_interm_layers)

In [5]:
def load_new_model(model_args, device, verbose=False,
                   pretrain_layers=['backbone', 'fpn', 'classification', 'regression'],
                   freeze_layers=[]):
    num_classes = 1

    backbone = CustomVGGBackboneWrapper(model_args.backbone)
    model = P2PNet(backbone, model_args.row, model_args.line)
    # load pretrained weights
    if len(pretrain_layers) > 0:
        checkpoint = torch.load('../CrowdCounting-P2PNet/weights/SHTechA.pth', map_location='cpu')
        valid = lambda s: s[:s.find('.')] in pretrain_layers
        valid_state = OrderedDict([(k,v) for k,v in checkpoint['model'].items() if valid(k)])
        missing, unexpected = model.load_state_dict(valid_state, strict=False)
        if verbose:
            print(f"pretrained weights loaded without the following keys:\n{'\n'.join(missing)}")
            if len(unexpected) > 0:
                print(f"WARNING: received unexpected keys:\n{'\n'.join(unexpected)}")
    for layer in freeze_layers:
        for param in tqdm(getattr(model, layer).parameters(), desc=layer):
            param.requires_grad = False


    weight_dict = {'loss_ce': 1, 'loss_points': model_args.point_loss_coef}
    losses = ['labels', 'points']
    matcher = build_matcher_crowd(model_args)
    criterion = SetCriterion_Crowd(num_classes, \
                                matcher=matcher, weight_dict=weight_dict, \
                                eos_coef=model_args.eos_coef, losses=losses)
    return model.to(device), criterion.to(device)

In [71]:
DEFAULT_TRANSFORM = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),
])

TRAIN_TRANSFORM = transforms.Compose([
    transforms.ToTensor(),
    # transforms.RandomHorizontalFlip(),
    # transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),
])
class GrapeDataset(Dataset):
    def __init__(self, root_path='labeled/', img_ext='.jpg', lbl_ext='_lbl.txt', \
                 transform=DEFAULT_TRANSFORM, train_transform=TRAIN_TRANSFORM, train=False, patch=False, flip=False):
        self.root = root_path + ('' if root_path[-1] == '/' else '/')
        self.instances = [f[:-len(lbl_ext)] for f in os.listdir(self.root)
                          if os.path.isfile(self.root+f) and f.endswith(lbl_ext)]
        self.img_ext = img_ext
        self.lbl_ext = lbl_ext
        self.trf = transform
        self.trn_trf = train_transform
        self.train = train
        self.patch = patch
        self.flip = flip
    
    def __len__(self):
        return len(self.instances)
    
    def _get_points(self, name):
        label_path = self.root + name + self.lbl_ext
        with open(label_path, 'r') as f:
            coords_list = f.readlines()
        coords = torch.tensor(tuple(tuple(map(float, line.split())) for line in coords_list))
        return coords
    
    def _load_image(self, name):
        img_path = self.root + name + self.img_ext
        img = Image.open(img_path)
        return img
    
    def __getitem__(self, i):
        # only support single-instance indexing
        img = self._load_image(self.instances[i]) # PIL image
        point = self._get_points(self.instances[i]) # tensor
        # perform val transforms on Image object
        if not (self.train and self.patch): # if not cropping, handle resizing
            # fix size of tensors (need to fix this)
            # HACK: only batch one example at a time
            width, height = img.size
            new_width = width // 128 * 128
            new_height = height // 128 * 128
            img = img.resize((new_width, new_height), Image.LANCZOS)
            point[:,0] *= new_width/width
            point[:,1] *= new_height/height
        # perform standard transforms
        if self.train and self.trn_trf is not None:
            img = self.trn_trf(img)
        elif self.trf is not None:
            img = self.trf(img)
        # additional transformation if training
        if self.train:
            # data augmentation -> random scale
            scale_range = [0.7, 1.3]
            min_size = min(img.shape[1:])
            scale = random.uniform(*scale_range)
            # scale the image and points
            if scale * min_size > 128:
                img = torch.nn.functional.upsample_bilinear(img.unsqueeze(0), scale_factor=scale).squeeze(0)
                point *= scale
            # random crop augumentation
            if self.patch:
                # generates a batch of images -> we just want one
                imgs, points = random_crop(img, point, num_patch=1)
                img, point = torch.tensor(imgs[0]), points[0]
            # random flipping
            if random.random() > 0.5 and self.flip:
                # random flip
                img = transforms.functional.hflip(img)
                for i, _ in enumerate(point):
                    point[:, 0] = 128 - point[:, 0]
            
        img = img.type(torch.float32)
        # pack up related infos
        img_id = torch.tensor(i, dtype=torch.int32, requires_grad=False)
        target = {
            'point': point,
            'image_id': img_id,
            'labels': torch.ones([point.shape[0]]).long()
        }
        return img, target

In [72]:
dl_kwargs = {
    # 'batch_size': 128,
    'num_workers': 0,
    'pin_memory': True,
    'collate_fn': utils.collate_fn
}
def get_subset_dataloaders(dset, subset_idx, k, bsz=128, dl_kwargs=dl_kwargs):
    val_idx = subset_idx[k]
    val_subset = Subset(dset, val_idx)
    val_dl = DataLoader(val_subset, batch_size=1, **dl_kwargs) # HACK
    train_idx = torch.cat((subset_idx[:k].flatten(), subset_idx[k+1:].flatten()))
    trn_subset = Subset(dset, train_idx)
    train_dl = DataLoader(trn_subset, shuffle=True, batch_size=bsz, **dl_kwargs)
    trn_evaldl = DataLoader(trn_subset, batch_size=1, **dl_kwargs)
    return train_dl, trn_evaldl, val_dl

In [88]:
hparams = EasyDict(
    lr_gen = 1e-4, # orig: 1e-4
    lr_backbone = 1e-5, # orig: 1e-5
    weight_decay = 1e-4, #orig: 1e-4
    epochs = 300,
    lr_drop = 1000,
    bsz = 64, # orig: 32
    clip_max_norm = 0.5, #orig: 0.1
    val_every = 10,
    pretrain_layers = ['backbone', 'fpn'],
    freeze_layers = []
)
model_args = EasyDict(
    # required to build inference model
    backbone = 'vgg16_bn', # name of the convolutional backbone to use
    pretrained = False,
    row = 1, # row number of anchor points
    line = 1, # line number of anchor points
    # required for model training
    point_loss_coef = 1e-3, # orig: 2e-4
    eos_coef = 0.5, #orig: 0.5 # Relative classification weight of the no-object class
    set_cost_class = 2, #orig: 1 # for matcher -- Class coefficient in the matching cost
    set_cost_point = .05 #orig: .05 # for matcher -- L1 point coefficient in the matching cost
)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
base_dset = GrapeDataset(train=False, patch=False, flip=False)
K = 5
print(len(base_dset))
assert len(base_dset) % K == 0 # kfold splits must be exactly even

200


In [89]:
seed_all()
folds = torch.randperm(len(base_dset)).reshape(K, -1)
# wipe vis directory for val outputs
for f in os.listdir(VIS_DIR):
    if f.endswith('jpg'):
        os.remove(f"{VIS_DIR}/{f}")
for k in range(K-1,K):
    print(f">>> FOLD {k+1}/{K} <<<")
    train_dl, trn_evaldl, val_dl = get_subset_dataloaders(base_dset, folds, k, bsz=hparams.bsz, dl_kwargs=dl_kwargs)
    model, criterion = load_new_model(model_args, device, pretrain_layers=hparams.pretrain_layers, freeze_layers=hparams.freeze_layers)
    opt_params = [
        {
            "params": [p for n, p in model.named_parameters() if "backbone" not in n and p.requires_grad],
            "lr": hparams.lr_gen
        },
        {
            "params": [p for n, p in model.named_parameters() if "backbone" in n and p.requires_grad],
            "lr": hparams.lr_backbone,
        },
    ]
    optimizer = torch.optim.Adam(opt_params)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, hparams.lr_drop)
    for attr in ['train', 'patch', 'flip']:
        base_dset.__setattr__(attr, True)
    for epoch in range(hparams.epochs):
        print(f"EPOCH {epoch+1}/{hparams.epochs}")
        stats = train_one_epoch(model, criterion, train_dl, optimizer, device, \
                                epoch, hparams.clip_max_norm)
        lr_scheduler.step()
        if epoch and (epoch+1) % hparams.val_every == 0:
            # HACK: disable and then re-enable training-time args for the dataset
            for attr in ['train', 'patch', 'flip']:
                base_dset.__setattr__(attr, False)
            # visualize final results only
            vis_dir = VIS_DIR if hparams.val_every + epoch >= hparams.epochs else None
            t_mae, t_mse = evaluate_crowd_no_overlap(model, trn_evaldl, device, vis_dir=vis_dir)
            print(f"trn: mae={t_mae}, mse={t_mse}")
            v_mae, v_mse = evaluate_crowd_no_overlap(model, val_dl, device, vis_dir=vis_dir)
            print(f"val: mae={v_mae}, mse={v_mse}")
            # reset stuff for training
            for attr in ['train', 'patch', 'flip']:
                base_dset.__setattr__(attr, True)
    print("val idxs:", sorted(folds[k].tolist()))
    # input('waiting for check >')


>>> FOLD 5/5 <<<
EPOCH 1/300


/var/folders/cp/ps4v9yh52dj0wy2dnwf912cr0000gn/T/ipykernel_94916/3981029517.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_id = torch.tensor(i, dtype=torch.int32, requires_grad=False)


Averaged stats: lr: 0.000100  loss: 0.6198 (0.6289)  loss_ce: 0.6198 (0.6289)  loss_ce_unscaled: 0.6198 (0.6289)  loss_point_unscaled: 12.8535 (12.8852)
EPOCH 2/300
Averaged stats: lr: 0.000100  loss: 0.4434 (0.4339)  loss_ce: 0.4434 (0.4339)  loss_ce_unscaled: 0.4434 (0.4339)  loss_point_unscaled: 13.6859 (13.5462)
EPOCH 3/300
Averaged stats: lr: 0.000100  loss: 0.3649 (0.3502)  loss_ce: 0.3649 (0.3502)  loss_ce_unscaled: 0.3649 (0.3502)  loss_point_unscaled: 14.6597 (14.8278)
EPOCH 4/300
Averaged stats: lr: 0.000100  loss: 0.3758 (0.3749)  loss_ce: 0.3758 (0.3749)  loss_ce_unscaled: 0.3758 (0.3749)  loss_point_unscaled: 14.8078 (14.8119)
EPOCH 5/300
Averaged stats: lr: 0.000100  loss: 0.3498 (0.3478)  loss_ce: 0.3498 (0.3478)  loss_ce_unscaled: 0.3498 (0.3478)  loss_point_unscaled: 14.8755 (14.9660)
EPOCH 6/300
Averaged stats: lr: 0.000100  loss: 0.3454 (0.3400)  loss_ce: 0.3454 (0.3400)  loss_ce_unscaled: 0.3454 (0.3400)  loss_point_unscaled: 14.6324 (14.6164)
EPOCH 7/300
Averaged s

In [96]:
model.load_state_dict(torch.load('best-weights.pth'))

<All keys matched successfully>

In [99]:
for attr in ['train', 'patch', 'flip']:
    base_dset.__setattr__(attr, False)
_, _, val_dl = get_subset_dataloaders(base_dset, folds, 4, bsz=hparams.bsz, dl_kwargs=dl_kwargs)
v_mae, v_mse = evaluate_crowd_no_overlap(model, val_dl, device, vis_dir=VIS_DIR)
print(f"val: mae={v_mae}, mse={v_mse}")

/var/folders/cp/ps4v9yh52dj0wy2dnwf912cr0000gn/T/ipykernel_94916/3981029517.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_id = torch.tensor(i, dtype=torch.int32, requires_grad=False)


val: mae=6.525, mse=9.374166629626338


In [58]:
torch.save(model.state_dict(), 'temp_best_mae9425.pth')

In [78]:
val_idxs = sorted(folds[k].tolist())
for f in os.listdir(VIS_DIR):
    if not os.path.isfile(f):
        continue
    f_idx = int(f[:f.find('_')])
    if not (f_idx in val_idxs):
        os.remove(f"{VIS_DIR}/{f}")

# fixing crops

In [38]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [39]:
path = 'cropped/'
images = [f for f in os.listdir(path)
          if os.path.isfile(path+f) and f.endswith('.jpg')]
THRESH = 5 # need more than THRESH nonzero values to be considered valid
BLACK_THRESH = 100
MIN_IMG_DIM = 128

# finds the first index of 1D array x that surpasses the threshold
find_first = lambda x: (x > THRESH).argmax()
for img_name in images:
    img = cv2.imread(path+img_name)
    nonblack_mask = (img > BLACK_THRESH)
    img_1ch = nonblack_mask.any(axis=2).astype(int) # flatten channels
    first_good_x = find_first(img_1ch.sum(axis=0))
    first_good_y = find_first(img_1ch.sum(axis=1))
    # crop
    cropped_img = img
    if first_good_x:
        cropped_img = cropped_img[:, first_good_x:-first_good_x]
    if first_good_y:
        cropped_img = cropped_img[first_good_y:-first_good_y]
    roi = cropped_img
    min_dim = min(roi.shape[0], roi.shape[1])
    scale = max(1, MIN_IMG_DIM/min_dim)
    scale_up = lambda l: int(np.ceil(scale*l))
    roi_out = cv2.resize(roi, (scale_up(roi.shape[1]), scale_up(roi.shape[0])))
    # fix coordinates
    # label_path = path+img_name[:-len("anno.jpg")]+"lbl.txt"
    # with open(label_path, 'r') as f:
    #     coords_list = f.readlines()
    #     coords = [tuple(map(int, line.split())) for line in coords_list]
    #     fixed_coords = [(scale_up(x-first_good_x), scale_up(y-first_good_y)) for (x,y) in coords]
    # label_str = '\n'.join([f"{x} {y}" for (x,y) in fixed_coords])
    # overwrite files
    cv2.imwrite(path+img_name, roi_out)
    # with open(label_path, 'w') as f:
    #     f.write(label_str)